<a href="https://colab.research.google.com/github/dakilaledesma/NCBG/blob/main/train/Multilabel_Parts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%time
! unzip -q /content/drive/MyDrive/UNC/NCBG/sept1_classif_training.zip -d /content
! cp /content/drive/MyDrive/UNC/NCBG/qry_fsus_trainingdata_ai.xlsx /content/qry_fsus_trainingdata_ai.xlsx

CPU times: user 44.8 ms, sys: 17.2 ms, total: 62 ms
Wall time: 6.35 s


In [2]:
! mv /content/content/images /content/images
! rm -rf /content/content

In [3]:
import pandas as pd

df = pd.read_excel("qry_fsus_trainingdata_ai.xlsx")

In [4]:
print(df.columns)

Index(['file', 'directory', 'imagequality', 'image_context'], dtype='object')


In [5]:
classes = list(set(sum([v.split(';') for v in df["image_context"] if type(v) == str], [])))
classes.sort()
class_dict = {v: i for i, v in enumerate(classes)}

In [6]:
print(class_dict)

{'aspect': 0, 'bud': 1, 'fall-color': 2, 'flower': 3, 'fruit': 4, 'leaf': 5, 'leaf-underside': 6, 'other': 7, 'root': 8, 'seed': 9, 'side-by-side': 10, 'stem': 11, 'venation': 12, 'vestiture': 13, 'whole-plant': 14}


In [7]:
import numpy as np

im_to_class = {}
for idx, row in df.iterrows():
  anno_bn = row["file"]
  try:
    classes = row["image_context"].split(';')
  except AttributeError:
    continue
  
  out_anno = np.zeros(15, dtype=int)
  for class_ in classes:
    class_idx = class_dict.get(class_)
    out_anno[class_idx] = 1
  
  im_to_class[anno_bn] = out_anno

In [8]:
from glob import glob
import os
from PIL import Image, UnidentifiedImageError
from tqdm.notebook import tqdm

X = []
y = []
for im_fn in glob("images/*.*"):
  bn = os.path.basename(im_fn).split("__")[1].replace(".jpg", '')
  bn = f"{bn}.jpg"
  try:
    y.append(im_to_class[bn])
    im = Image.open(im_fn)
    im = im.resize((331, 331))
    im = np.array(im, dtype=int)
  except KeyError or UnidentifiedImageError:
    continue

  X.append(im)

In [9]:
print(len(X), len(y))

8489 8489


In [10]:
X = np.array(X, dtype=int)
y = np.array(y, dtype=int)

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import NASNetLarge
from tensorflow.keras.layers import Dense

nasnet = NASNetLarge(input_shape=(331, 331, 3), include_top=False, pooling='avg')
model = Sequential()
model.add(nasnet)
model.add(Dense(15, activation="sigmoid"))

343621632/343610240 [==============================] - 7s 0us/step


In [12]:
print(X.shape, y.shape)

(8489, 331, 331, 3) (8489, 15)


In [13]:
gen = tf.keras.preprocessing.image.ImageDataGenerator()
res = gen.flow(X, y)
x1, y1 = next(res)

In [14]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics="acc")
model.fit(x=x1, y=y1, batch_size=16, epochs=80)

Epoch 1/80
2/2 [==============================] - 42s 900ms/step - loss: 0.5566 - acc: 0.3750
Epoch 2/80
2/2 [==============================] - 2s 899ms/step - loss: 0.1957 - acc: 0.6562
Epoch 3/80
2/2 [==============================] - 2s 924ms/step - loss: 0.0759 - acc: 0.8125
Epoch 4/80
2/2 [==============================] - 2s 909ms/step - loss: 0.0145 - acc: 0.8125
Epoch 5/80
2/2 [==============================] - 2s 914ms/step - loss: 0.0033 - acc: 0.7812
Epoch 6/80
2/2 [==============================] - 2s 928ms/step - loss: 0.0019 - acc: 0.7500
Epoch 7/80
2/2 [==============================] - 2s 931ms/step - loss: 0.0011 - acc: 0.8125
Epoch 8/80
2/2 [==============================] - 2s 948ms/step - loss: 8.2322e-04 - acc: 0.7812
Epoch 9/80
2/2 [==============================] - 2s 923ms/step - loss: 5.4925e-04 - acc: 0.7188
Epoch 10/80
2/2 [==============================] - 2s 922ms/step - loss: 2.2682e-04 - acc: 0.7188
Epoch 11/80
2/2 [==============================] - 2s 92

In [15]:
model.save("/content/drive/MyDrive/UNC/NCBG/mp_1")